In [1]:
!npm install jupnode
import jupnode


> jupnode@0.5.73 install /content/node_modules/jupnode
> node ./install

Processing ./jupnode-0.0.0.tar.gz
  Created wheel for jupnode: filename=jupnode-0.0.0-cp37-none-any.whl size=267839 sha256=083fc7e415d2a66eb19c6f3e98da59affb816012b6ec159883e6a386b58927ac
  Stored in directory: /root/.cache/pip/wheels/cb/2b/76/03debf4dac1dd365dc0362fe7beca25a42fa452d4a3bf8e9ef
Successfully built jupnode

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ jupnode@0.5.73
added 1 package from 1 contributor and audited 1 package in 4.385s
found 0 vulnerabilities



In [2]:
sh('git clone https://github.com/Graphnull/enheg');
sh('npm i sharp');
sh(`npm i @tensorflow/tfjs-node-gpu`);

Cloning into 'enheg'...
> sharp@0.28.3 install /content/node_modules/sharp
> (node install/libvips && node install/dll-copy && prebuild-install) || (node install/can-compile && node-gyp rebuild && node install/dll-copy)
sharp: Downloading https://github.com/lovell/sharp-libvips/releases/download/v8.10.6/libvips-8.10.6-linux-x64.tar.br
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.
+ sharp@0.28.3
added 68 packages from 192 contributors and audited 69 packages in 4.504s
6 packages are looking for funding
run `npm fund` for details
found 0 vulnerabilities
> @tensorflow/tfjs-node-gpu@3.7.0 install /content/node_modules/@tensorflow/tfjs-node-gpu
> node scripts/install.js gpu download
GPU-linux-3.7.0.tar.gz
* Downloading libtensorflow
* Buildi

In [14]:
let tf = require('@tensorflow/tfjs-node-gpu')


let model = null;
//try{
//  model = await tf.loadLayersModel('file://./model/model.json');
//}catch(err){
  //  console.log(err)
  model = tf.sequential();
  model.add(tf.layers.conv2d({ filters: 128, kernelSize: [9, 9], activation: 'linear', padding: 'same', useBias: true, inputShape: [null,null, 1]}));
  model.add(tf.layers.leakyReLU());
  model.add(tf.layers.conv2d({ filters: 64, kernelSize: [5, 5], activation: 'linear', padding: 'same', useBias: true }));
  model.add(tf.layers.leakyReLU());
  model.add(tf.layers.conv2d({ filters: 32, kernelSize: [3, 3], activation: 'linear', padding: 'same', useBias: true }));
  model.add(tf.layers.leakyReLU());
  model.add(tf.layers.conv2d({ filters: 32, kernelSize: [3, 3], activation: 'linear', padding: 'same', useBias: true }));
  model.add(tf.layers.activation({activation: 'relu6'}));
  model.add(tf.layers.conv2d({ filters: 1, kernelSize: [3, 3], activation: 'linear', padding: 'same', useBias: true }));
  model.add(tf.layers.leakyReLU());
//}
model.summary();


_________________________________________________________________
Layer (type)                 Output shape              Param #
conv2d_Conv2D16 (Conv2D)     [null,null,null,128]      10496
_________________________________________________________________
leaky_re_lu_LeakyReLU16 (Lea [null,null,null,128]      0
_________________________________________________________________
conv2d_Conv2D17 (Conv2D)     [null,null,null,64]       204864
_________________________________________________________________
leaky_re_lu_LeakyReLU17 (Lea [null,null,null,64]       0
_________________________________________________________________
conv2d_Conv2D18 (Conv2D)     [null,null,null,32]       18464
_________________________________________________________________
leaky_re_lu_LeakyReLU18 (Lea [null,null,null,32]       0
_________________________________________________________________
conv2d_Conv2D19 (Conv2D)     [null,null,null,32]       9248
____________________________________________________________

In [17]:
let generateDataset = require('./enheg/dataset_generate/index')

let loss = (result, output) => {
    let start = [0, 16, 16, 0];
    let end = [result.shape[0], result.shape[1]-32, result.shape[2]-32, 1];
  return tf.pow(result.sub(output), 2)
  .slice(start, end)
  .mean();
}

model.compile({loss, optimizer: tf.train.adam(0.00001)});

for(let i=0;i!==100;i++){
    
    let { result, lines, width, height} = await generateDataset()
    let [x, y] = tf.tidy(()=>{
        let x = tf.tensor(result,[1,height/2, width/2,1]).div(255)
        let y = tf.tensor(lines,[1,height/2, width/2,1]).div(255)
      return [x, y]
    })
    
    let loss = await model.trainOnBatch (x, y)
    console.log(loss);
    x.dispose()
    y.dispose()
    
}




0.005379193462431431

0.01062831375747919

0.0067602964118123055

0.006160244345664978

0.004389761947095394

0.004602991044521332

0.005296221934258938

0.006149146240204573

0.005243709776550531

0.004129026085138321

0.004705223720520735

0.010909659788012505

0.004940909333527088

0.006218838971108198

0.006531533785164356

0.004851779900491238

0.007370542269200087

0.011510654352605343

0.00869839359074831

0.004597460851073265

0.005532768554985523

0.004371343180537224

0.005166751332581043

0.005593631416559219

0.007957161404192448

0.006552577018737793

0.004700528457760811

0.0049538505263626575

0.004767747595906258

0.004547858610749245

0.008279027417302132

0.00807182863354683

0.004808067809790373

0.004947110544890165

0.004383542574942112

0.004720297642052174

0.006626555696129799

0.004821358248591423

0.004252876620739698

0.005432016681879759

0.004664058331400156

0.010449804365634918

0.005473668687045574

0.004283275455236435

0.00756117282435298

0.0049413405

In [18]:
await model.save('file://model');

{
modelArtifactsInfo: {
dateSaved: 2021-06-17T08:16:40.865Z,
modelTopologyType: 'JSON',
modelTopologyBytes: 3405,
weightSpecsBytes: 676,
weightDataBytes: 973444
}
}
